ПАРСИНГ


In [ ]:
import requests
import pandas as pd
import csv
from datetime import datetime
import re

def take_1000_posts():
    token = 'a2efe549a2efe549a2efe54955a1fc4b8faa2efa2efe549c6ae4dd24dc2e9d34ffe1aa4'
    version = 5.92
    domain = 'foodbankrus'
    count = 100
    offset = 0
    all_posts = []


    while offset < 1000:
        response = requests.get('https://api.vk.com/method/wall.get',
                            params={
                                'access_token': token,
                                'v': version,
                                'domain': domain,
                                'count': count,
                                'offset': offset
                            }
                            )
        data = response.json()['response']['items']
        offset += count
        all_posts.extend(data)
    return all_posts

def file_writer(data):
    with open('Parsing_nice.csv', 'w', encoding='utf8') as file:
        dataset = csv.writer(file)
        dataset.writerow(('date', 'text', 'likes', 'url'))
        for post in data:
            date = ''
            date = datetime.utcfromtimestamp(post['date']).strftime('%Y-%m-%d %H:%M:%S')

            img_url = ''
            try:
                if post['attachments'][0]['type']:
                    img_url = post['attachments'][0]['photo']['sizes'][-1]['url']
                else:
                    img_url = 'pass'
            except:
                pass

            dataset.writerow((date, post['text'], post['likes']['count'], img_url))


all_posts = take_1000_posts()
file_writer(all_posts)


In [ ]:
datasetread = pd.read_csv('Parsing_nice.csv', sep=',')

In [ ]:
datasetread

ОБРАБОТКА ТЕКСТА


In [ ]:
!pip -V

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 58.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=1c509904a9b9d791fcc5cc7572bf1ecf73b8ebaddd791a954e29f04ec2cf21af
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import chardet
import re

In [ ]:
# Функция возвращяющая dataframe с некорректными символами содержащимися в столбце
def get_illegal_characters(datasetread, column_name, legitimate_characters, encoding='utf8', DEBUG=False):

  # получим все не легитимные символы из датафрейма
  result = datasetread[column_name].str.extractall(legitimate_characters)

  # Добавим столбец с hex кодировкой символов.
  result['hex_code'] = result[0].str.encode(encoding)

  # Дадим адекватные имена столбцам.
  result.rename(columns={0: 'character'}, inplace=True)

  if (DEBUG):
    print("Найдено", result.shape[0], "уникальных символов не подходящих под заданный шаблон.")
    print("Найдены следующие символы:\n", result)

  return result
# Шаблон "легитимных символов". Использование этих символов допустимо в русском языке.
# Естественно символ пробела мы тоже оставим, иначе наши слова "склеятся".
legitimate_characters = "([^а-яёА-ЯЁ ])"

# Имя столбца, который необходимо проверить
column_name = 'text'

bad_symbols =  get_illegal_characters(datasetread, column_name, legitimate_characters)

bad_symbols

character hex_code
    match                   
0   0             2     b'2'
    1             0     b'0'
    2             1     b'1'
    3             2     b'2'
    4            \n    b'\n'
...             ...      ...
999 45            w     b'w'
    46            a     b'a'
    47            s     b's'
    48            t     b't'
    49            e     b'e'

[83950 rows x 2 columns]

In [ ]:
# Создадим шаблон из найденных ненужных символов.
# Ненужные символы необходимо предварительно экранировать, затем обьединить
# в список.
template = '|'.join(map(re.escape, bad_symbols['character']))

# Удалим ненужные символы из новостей.
datasetread['text'] = datasetread['text'].str.replace(template, '', regex=True, flags=re.UNICODE)

# Посмотрим на результат
datasetread

,date,text,likes,url
0,2023-04-05 15:09:41,С года мы развиваем фудшеринг в странеПринима...,268,NaN
1,2023-05-22 08:23:21,Подписали соглашение о сотрудничестве с Ворон...,99,NaN
2,2023-05-19 13:51:23,Дарите подарки близким и помогайте нуждающимся...,136,https://sun1-56.userapi.com/impg/Wl2fzIDDC620j...
3,2023-05-18 16:23:04,В России каждый год рестораны и магазины выбра...,258,NaN
4,2023-05-17 17:53:59,Корзин Доброты для одиноких пожилых людей и ...,407,https://sun1-92.userapi.com/impg/ay5_4eLACyeWW...
...,...,...,...,...
995,2019-01-14 16:50:09,Интересные факты о молокеУченые считают что мо...,44,https://sun9-42.userapi.com/impf/c830409/v8304...
996,2019-01-14 11:46:56,Нет топлива нет еды Ежедневно мы доставляем ...,38,https://sun9-71.userapi.com/impf/c847217/v8472...
997,2019-01-12 16:23:24,Итоги ежегодного исследования выдающихся люде...,82,https://sun9-77.userapi.com/impf/c850628/v8506...
998,2019-01-11 11:12:10,Уже завтра января состоится первая в этом год...,45,https://sun9-41.userapi.com/impf/c846320/v8463...


In [ ]:
Text = datasetread.iloc[0]['text']

In [ ]:
Text

'С  года мы развиваем фудшеринг в странеПринимаем невостребованные продукты у бизнеса и передаем их людям Развиваем национальную сеть банков еды Применяем технологии товарной благотворительности Обучаем волонтеров проводим конференции и курсы Делимся опытом и знаниями чтобы каждый мог рационально использовать ресурсы нашей планеты  Больше  лет банк еды продолжает следовать своей миссии Но мы не стоим на месте Поменяли название Стали звучать и выглядеть поновому  Теперь мы запускаем масштабную рекламную кампанию  чтобы больше людей узнали про фудшеринг и задумались о бережном обращении к продуктам Запустить кампанию нам помогает  наш стратегический партнер Благодаря ему социальную рекламу банка еды Русь можно увидеть на федеральных телеканалах и билбордах в Москве Нижнем Новгороде Краснодаре Волгограде Екатеринбурге Перми Омске Хабаровске Уфе Новосибирске Красноярске и Кургане  Поддерживать развитие технологии фудшеринга  значит повышать общий уровень благополучия в стране и качество св

In [ ]:
tokens = word_tokenize(Text.lower())
tokens

In [ ]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...']

In [ ]:
only_words = []
for token in tokens:
    if token not in punctuation_marks:
        only_words.append(token)

only_words

In [ ]:
morph = pymorphy2.MorphAnalyzer()

lemmas = []
for token in only_words:
    lemmas.append(morph.parse(token)[0].normal_form)

lemmas

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words("russian")
stop_words

In [ ]:
filtered_words = []
for token in lemmas:
    if token not in stop_words:
        filtered_words.append(token)

filtered_words

In [ ]:
def preprocess(Text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(Text.lower())
    preprocessed_Text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_Text.append(lemma)
    return preprocessed_Text


punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...']
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()


In [ ]:
datasetread = datasetread.dropna (subset=['text'])

In [ ]:
datasetread['Preprocessed_texts'] = datasetread.apply(lambda row: preprocess(row['text'], punctuation_marks, stop_words, morph), axis=1)

<ipython-input-18-ba68e79c1c66>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasetread['Preprocessed_texts'] = datasetread.apply(lambda row: preprocess(row['text'], punctuation_marks, stop_words, morph), axis=1)


In [ ]:
datasetread

,date,text,likes,url,Preprocessed_texts
0,2023-04-05 15:09:41,С года мы развиваем фудшеринг в странеПринима...,252,NaN,"[год, развивать, фудшеринг, странепринимать, н..."
1,2023-05-04 16:14:57,Делать жизнь людей лучше каждый день это тоже...,84,NaN,"[делать, жизнь, человек, каждый, день, это, не..."
2,2023-05-03 12:23:53,В России ежегодно выбрасывается на свалку боле...,129,NaN,"[россия, ежегодно, выбрасываться, свалка, млн,..."
3,2023-04-27 17:37:30,Посмотрите какое классное видео сделали наши д...,88,NaN,"[посмотреть, какой, классный, видео, сделать, ..."
4,2023-04-26 17:03:02,Мы запускаем свое собственное медиа Банк еды Р...,41,https://sun9-70.userapi.com/impg/J45_DhqCrZjz-...,"[запускать, свой, собственный, медиа, банк, ед..."
...,...,...,...,...,...
995,2019-01-04 17:11:58,Франция и Чехия первые страны в мире которые ...,104,https://sun9-70.userapi.com/impf/c850520/v8505...,"[франция, чехия, первый, страна, мир, который,..."
996,2019-01-03 16:52:12,Достучаться до сердецКаждый год глухой одиноки...,200,https://sun9-56.userapi.com/impf/A7XNrQSRbYjYO...,"[достучаться, сердецкаждый, год, глухой, одино..."
997,2019-01-02 15:33:55,Мощный заряд новогоднего настроения с любимыми...,86,NaN,"[мощный, заряд, новогодний, настроение, любимы..."
998,2019-01-01 15:49:24,Новый год прошел а впереди еще большие новогод...,52,https://sun9-47.userapi.com/impf/c849136/v8491...,"[новый, год, пройти, впереди, больший, новогод..."


In [ ]:
datasetread.to_csv (r' D:\Мои документы\Учёба\Курс 3\6 семестр\Курсач годовой\Контент\my_datasetread.csv', index= False, sep=",")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

Определяем частоту слов


In [ ]:
from collections import Counter

test_counter = Counter(datasetread['Preprocessed_texts'][0])
test_counter

In [ ]:
test_counter.most_common(10)

In [ ]:
words = Counter()

for txt in datasetread['Preprocessed_texts']:
    words.update(txt)

In [ ]:
len(words)

9554

In [ ]:
words.most_common(100)

In [ ]:
datasetread.dtypes

date                  object
text                  object
likes                  int64
url                   object
Preprocessed_texts    object
dtype: object

In [ ]:
date_cols = ["date"]
datasetread[date_cols] = datasetread[date_cols].astype("datetime64[ns]")
datasetread

<ipython-input-31-770cf613bebf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasetread[date_cols] = datasetread[date_cols].astype("datetime64[ns]")


,date,text,likes,url,Preprocessed_texts
0,2023-04-05 15:09:41,С года мы развиваем фудшеринг в странеПринима...,252,NaN,"[год, развивать, фудшеринг, странепринимать, н..."
1,2023-05-04 16:14:57,Делать жизнь людей лучше каждый день это тоже...,84,NaN,"[делать, жизнь, человек, каждый, день, это, не..."
2,2023-05-03 12:23:53,В России ежегодно выбрасывается на свалку боле...,129,NaN,"[россия, ежегодно, выбрасываться, свалка, млн,..."
3,2023-04-27 17:37:30,Посмотрите какое классное видео сделали наши д...,88,NaN,"[посмотреть, какой, классный, видео, сделать, ..."
4,2023-04-26 17:03:02,Мы запускаем свое собственное медиа Банк еды Р...,41,https://sun9-70.userapi.com/impg/J45_DhqCrZjz-...,"[запускать, свой, собственный, медиа, банк, ед..."
...,...,...,...,...,...
995,2019-01-04 17:11:58,Франция и Чехия первые страны в мире которые ...,104,https://sun9-70.userapi.com/impf/c850520/v8505...,"[франция, чехия, первый, страна, мир, который,..."
996,2019-01-03 16:52:12,Достучаться до сердецКаждый год глухой одиноки...,200,https://sun9-56.userapi.com/impf/A7XNrQSRbYjYO...,"[достучаться, сердецкаждый, год, глухой, одино..."
997,2019-01-02 15:33:55,Мощный заряд новогоднего настроения с любимыми...,86,NaN,"[мощный, заряд, новогодний, настроение, любимы..."
998,2019-01-01 15:49:24,Новый год прошел а впереди еще большие новогод...,52,https://sun9-47.userapi.com/impf/c849136/v8491...,"[новый, год, пройти, впереди, больший, новогод..."


In [ ]:
datasetread["year"] = datasetread["date"].dt.to_period("Y")
datasetread

<ipython-input-32-490bfafef25a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasetread["year"] = datasetread["date"].dt.to_period("Y")


,date,text,likes,url,Preprocessed_texts,year
0,2023-04-05 15:09:41,С года мы развиваем фудшеринг в странеПринима...,252,NaN,"[год, развивать, фудшеринг, странепринимать, н...",2023
1,2023-05-04 16:14:57,Делать жизнь людей лучше каждый день это тоже...,84,NaN,"[делать, жизнь, человек, каждый, день, это, не...",2023
2,2023-05-03 12:23:53,В России ежегодно выбрасывается на свалку боле...,129,NaN,"[россия, ежегодно, выбрасываться, свалка, млн,...",2023
3,2023-04-27 17:37:30,Посмотрите какое классное видео сделали наши д...,88,NaN,"[посмотреть, какой, классный, видео, сделать, ...",2023
4,2023-04-26 17:03:02,Мы запускаем свое собственное медиа Банк еды Р...,41,https://sun9-70.userapi.com/impg/J45_DhqCrZjz-...,"[запускать, свой, собственный, медиа, банк, ед...",2023
...,...,...,...,...,...,...
995,2019-01-04 17:11:58,Франция и Чехия первые страны в мире которые ...,104,https://sun9-70.userapi.com/impf/c850520/v8505...,"[франция, чехия, первый, страна, мир, который,...",2019
996,2019-01-03 16:52:12,Достучаться до сердецКаждый год глухой одиноки...,200,https://sun9-56.userapi.com/impf/A7XNrQSRbYjYO...,"[достучаться, сердецкаждый, год, глухой, одино...",2019
997,2019-01-02 15:33:55,Мощный заряд новогоднего настроения с любимыми...,86,NaN,"[мощный, заряд, новогодний, настроение, любимы...",2019
998,2019-01-01 15:49:24,Новый год прошел а впереди еще большие новогод...,52,https://sun9-47.userapi.com/impf/c849136/v8491...,"[новый, год, пройти, впереди, больший, новогод...",2019


In [ ]:
list = datasetread["year"] == pd.Period('2023', freq='Y')
for i in list:
  if i == True:
    print(words.most_common(10))

# Хрень

In [ ]:
# Словарь, отображающий слова в коды
word_to_index = dict()
# Словарь, отображающий коды в слова
index_to_word = dict()

max_words = 10000

In [ ]:
for i, word in enumerate(words.most_common(max_words - 2)):
    word_to_index[word[0]] = i + 2
    index_to_word[i + 2] = word[0]

word_to_index

In [ ]:
index_to_word

Преобразование слов в коды

In [ ]:
def text_to_sequence(txt, word_to_index):
    seq = []
    for word in txt:
        index = word_to_index.get(word, 1) # 1 означает неизвестное слово
        # Неизвестные слова не добавляем в выходную последовательность
        if index != 1:
            seq.append(index)
    return seq

txt = datasetread['Preprocessed_texts'][0]
txt

In [ ]:
seq = text_to_sequence(txt, word_to_index)
seq

In [ ]:
index_to_word[241]

'вклад'

In [ ]:
seq = [word_to_index.get(word, 1) for word in txt]
seq

In [ ]:
datasetread['Sequences'] = datasetread.apply(lambda row: text_to_sequence(row['Preprocessed_texts'], word_to_index), axis=1)
datasetread

<ipython-input-41-61e97fd57b6d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasetread['Sequences'] = datasetread.apply(lambda row: text_to_sequence(row['Preprocessed_texts'], word_to_index), axis=1)


,date,text,likes,url,Preprocessed_texts,year,Sequences
0,2023-04-05 15:09:41,С года мы развиваем фудшеринг в странеПринима...,252,NaN,"[год, развивать, фудшеринг, странепринимать, н...",2023,"[13, 474, 43, 4726, 1421, 2, 203, 210, 9, 474,..."
1,2023-05-04 16:14:57,Делать жизнь людей лучше каждый день это тоже...,84,NaN,"[делать, жизнь, человек, каждый, день, это, не...",2023,"[95, 129, 9, 28, 17, 12, 343, 2412, 1425]"
2,2023-05-03 12:23:53,В России ежегодно выбрасывается на свалку боле...,129,NaN,"[россия, ежегодно, выбрасываться, свалка, млн,...",2023,"[19, 272, 775, 334, 151, 40, 2, 117, 71, 1889,..."
3,2023-04-27 17:37:30,Посмотрите какое классное видео сделали наши д...,88,NaN,"[посмотреть, какой, классный, видео, сделать, ...",2023,"[655, 352, 2120, 1694, 47, 7, 91, 2121, 402, 1..."
4,2023-04-26 17:03:02,Мы запускаем свое собственное медиа Банк еды Р...,41,https://sun9-70.userapi.com/impg/J45_DhqCrZjz-...,"[запускать, свой, собственный, медиа, банк, ед...",2023,"[602, 26, 497, 1891, 44, 18, 15, 777, 1426, 16..."
...,...,...,...,...,...,...,...
995,2019-01-04 17:11:58,Франция и Чехия первые страны в мире которые ...,104,https://sun9-70.userapi.com/impf/c850520/v8505...,"[франция, чехия, первый, страна, мир, который,...",2019,"[2457, 4723, 41, 78, 108, 5, 1872, 928, 188, 4..."
996,2019-01-03 16:52:12,Достучаться до сердецКаждый год глухой одиноки...,200,https://sun9-56.userapi.com/impf/A7XNrQSRbYjYO...,"[достучаться, сердецкаждый, год, глухой, одино...",2019,"[9524, 9525, 13, 9526, 46, 138, 890, 9527, 952..."
997,2019-01-02 15:33:55,Мощный заряд новогоднего настроения с любимыми...,86,NaN,"[мощный, заряд, новогодний, настроение, любимы...",2019,"[2781, 3447, 358, 1237, 404, 2890, 9543, 89, 4..."
998,2019-01-01 15:49:24,Новый год прошел а впереди еще большие новогод...,52,https://sun9-47.userapi.com/impf/c849136/v8491...,"[новый, год, пройти, впереди, больший, новогод...",2019,"[76, 13, 73, 1363, 134, 358, 1959, 479, 3267, ..."


In [ ]:
import numpy as np

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for index in sequence:
            results[i, index] += 1.
    return results


In [ ]:
vectors = vectorize_sequences(datasetread['Sequences'], max_words)

In [ ]:
vectors = vectorize_sequences(datasetread['Sequences'], max_words)

In [ ]:
vectors[0][:100]

In [ ]:
len(vectors[0])

10000

Обучение Word2Vec

In [ ]:
import gensim.downloader

In [ ]:
word2vec_eng = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
list(gensim.downloader.info()['models'].keys())

# Не поняла зачем

In [ ]:
word2vec_rus = gensim.downloader.load('word2vec-ruscorpora-300')

[=================================================-] 100.0% 198.8/198.8MB downloaded


In [ ]:
for index, word in enumerate(word2vec_rus.index_to_key ):
    if index == 20:
        break
    print(f"word #{index}/{len(word2vec_rus.index_to_key )} is {word}")

In [ ]:
word2vec_rus['еда_NOUN']

In [ ]:
word2vec_rus.most_similar(['еда_NOUN'])

In [ ]:
import gensim.models

In [ ]:
model = gensim.models.Word2Vec(sentences=datasetread['Preprocessed_texts'],
                               min_count=5,
                               vector_size=100)

In [ ]:
model.wv['еда']

In [ ]:
model.wv['волонтёр']

In [ ]:
model.wv.most_similar('продукт')

In [ ]:
model.wv.most_similar('фонд')

In [ ]:
model.wv.most_similar('волонтёр')

In [ ]:
model.wv.most_similar('помощь')

In [ ]:
model.wv.most_similar('наш')

In [ ]:
model.wv.most_similar('семья')

In [ ]:
model.wv.most_similar('человек')

In [ ]:
model.wv.most_similar('набор')

In [ ]:
model.wv.most_similar('нуждаться')

In [ ]:
model.wv.most_similar('год')

In [ ]:
model.wv.most_similar('экология')

In [ ]:
model.save('word2vec-food.ru-50')

In [ ]:
!ls -lh

total 7.1M
-rw-r--r-- 1 root root 3.3M May  5 12:01 ' D:\Мои документы\Учёба\Курс 3\6 семестр\Курсач годовой\Контент\my_datasetread.csv'
-rw-r--r-- 1 root root 1.9M May  5 11:59  Parsing_nice.csv
drwxr-xr-x 1 root root 4.0K May  3 13:31  sample_data
-rw-r--r-- 1 root root 2.0M May  5 12:14  word2vec-food.ru-50


In [ ]:
new_model = gensim.models.Word2Vec.load('word2vec-food.ru-50')

In [ ]:
new_model.wv['помощь']

Тематическое моделирование

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
!pip install pyLDAvis


In [ ]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


Помолимся

In [ ]:
!pip install spacy

# Run in python console
import nltk; nltk.download('stopwords')
import spacy

In [ ]:
import en_core_web_sm

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(datasetread['Preprocessed_texts'], min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[datasetread['Preprocessed_texts']], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[datasetread['Preprocessed_texts'][0]]])

['год', 'развивать', 'фудшеринг', 'странепринимать', 'невостребованный', 'продукт', 'бизнес', 'передавать', 'человек', 'развивать', 'национальный_сеть', 'банк_еда', 'применять', 'технология', 'товарный', 'благотворительность', 'обучать', 'волонтёр', 'проводить', 'конференция', 'курс', 'делиться', 'опыт', 'знание', 'каждый', 'мочь', 'рационально_использовать', 'ресурс', 'наш', 'планета', 'год', 'банк_еда', 'продолжать', 'следовать', 'свой', 'миссия', 'стоить', 'место', 'поменять', 'название', 'стать', 'звучать', 'выглядеть', 'поновое', 'запускать', 'масштабный', 'рекламный', 'кампания', 'человек', 'узнать', 'фудшеринг', 'задуматься', 'бережный', 'обращение', 'продукт', 'запустить', 'кампания', 'мы', 'помогать', 'наш', 'стратегический', 'партнёр', 'благодаря', 'социальный', 'реклама', 'банк_еда', 'русь', 'увидеть', 'федеральный', 'телеканал', 'билборд', 'москва', 'нижний_новгород', 'краснодар', 'волгоград', 'екатеринбург', 'пермь', 'омск', 'хабаровск', 'уфа', 'новосибирск', 'красноярск',

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(datasetread['Preprocessed_texts'])

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['развивать', 'странепринимать', 'продукт', 'бизнес', 'передавать', 'человек', 'развивать', 'национальный', 'сеть', 'курс', 'делиться', 'опыт', 'рационально_использовать', 'наш', 'планета', 'продолжать', 'следовать', 'стоить', 'звучать', 'поновое', 'человек', 'узнать', 'фудшеринг', 'задуматься', 'бережный', 'запустить', 'помогать', 'наш', 'стратегический', 'партнёр', 'благодаря', 'социальный', 'реклама', 'банк_еда', 'русь', 'нижний_новгород', 'красноярск', 'курган', 'поддерживать', 'развитие', 'технология', 'повышать', 'общий', 'передать', 'продукт', 'товар', 'нуждаться', 'допустить', 'выброс', 'свалка', 'помочь', 'сохранить', 'открыть', 'счёт', 'добрый_дело', 'вместе', 'банк_еда', 'русь', 'фудшеринг']]


Не помогло

In [ ]:
id2word = corpora.Dictionary(datasetread['Preprocessed_texts'])
texts = datasetread['Preprocessed_texts']
corpus = [id2word.doc2bow(text) for text in datasetread['Preprocessed_texts']]
print(corpus[:1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 4), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 4), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 2), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 3), (75, 2), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 2), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 4), (106, 1), (107, 3), (108, 1)]]


In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
lda_model.print_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.016*"фестиваль" + 0.013*"ответственный" + 0.009*"мастеркласс" + 0.009*"сообщение" + 0.009*"необычный" + 0.008*"полезный" + 0.008*"дарить" + 0.008*"оставлять" + 0.008*"вариант" + 0.007*"художник"'),
 (1,
  '0.024*"россия" + 0.017*"тонна" + 0.014*"компания" + 0.014*"кг" + 0.013*"средство" + 0.012*"мывместе" + 0.010*"помощь" + 0.009*"пожертвовать" + 0.009*"питание" + 0.009*"акция"'),
 (2,
  '0.043*"животное" + 0.038*"корм" + 0.036*"приют" + 0.032*"казань" + 0.019*"бездомный" + 0.012*"июль" + 0.011*"достаточно" + 0.011*"суббота" + 0.010*"собака" + 0.010*"кошка"'),
 (3,
  '0.032*"банан" + 0.015*"банановый" + 0.006*"кабачок" + 0.006*"порошок" + 0.006*"кусочек" + 0.006*"связка" + 0.004*"варить" + 0.003*"выпечка" + 0.003*"неделякабачок" + 0.003*"невыбрасывайкабачок"'),
 (4,
  '0.017*"наш" + 0.016*"день" + 0.012*"бабушка" + 0.012*"свой" + 0.010*"год" + 0.010*"еда" + 0.010*"помочь" + 0.010*"дедушка" + 0.010*"человек" + 0.009*"дом"'),
 (5,
  '0.023*"полезный" + 0.014*"йогурт" + 0.012*"п

In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

coherence_model_lda = CoherenceModel(model=lda_model, texts=datasetread['Preprocessed_texts'], dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -8.037011746616038

Coherence Score:  0.4587834760943359


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
ERROR:concurrent.futures:exception calling callback for <Future at 0x7fd9894e43a0 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py", line 391, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
ModuleNotFoundError: No module named 'pandas.core.indexes.numeric'
"""

The above exception was the direct cause of the following exc

BrokenProcessPool: ignored

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis
if __name__ == '__main__':
  vis


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
ERROR:concurrent.futures:exception calling callback for <Future at 0x7fd889229060 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py", line 391, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
ModuleNotFoundError: No module named 'pandas.core.indexes.numeric'
"""

The above exception was the direct cause of the following exc

BrokenProcessPool: ignored